In [1]:
import random
import numpy as np

class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            # computes the activation of the next layer
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), len(test_data)))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # computation of the gradient, summed over samples
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        # update step
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [2]:
import pickle
import gzip
import numpy as np

def load_data(path=None):
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.
    """
    if path is None:
        path = './data/mnist.pkl.gz'
    with gzip.open(path, 'rb') as f:
        training_data, validation_data, test_data = pickle.load(f, encoding='bytes')
    return (training_data, validation_data, test_data)

def load_data_wrapper(path=None):
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.
    """
    tr_d, va_d, te_d = load_data(path)
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = list(zip(training_inputs, training_results))
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = list(zip(validation_inputs, va_d[1]))
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = list(zip(test_inputs, te_d[1]))
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [3]:
training_data, validation_data, test_data = load_data_wrapper('neural-networks-and-deep-learning/data/mnist.pkl.gz')

In [4]:
net = Network([784, 30, 10])

In [5]:
%%time
net.SGD(training_data, 30, 10, 3.0, test_data=test_data )

Epoch 0: 9023 / 10000
Epoch 1: 9230 / 10000
Epoch 2: 9318 / 10000
Epoch 3: 9363 / 10000
Epoch 4: 9365 / 10000
Epoch 5: 9406 / 10000
Epoch 6: 9396 / 10000
Epoch 7: 9429 / 10000
Epoch 8: 9404 / 10000
Epoch 9: 9428 / 10000
Epoch 10: 9431 / 10000
Epoch 11: 9473 / 10000
Epoch 12: 9430 / 10000
Epoch 13: 9470 / 10000
Epoch 14: 9469 / 10000
Epoch 15: 9463 / 10000
Epoch 16: 9487 / 10000
Epoch 17: 9487 / 10000
Epoch 18: 9463 / 10000
Epoch 19: 9484 / 10000
Epoch 20: 9450 / 10000
Epoch 21: 9480 / 10000
Epoch 22: 9498 / 10000
Epoch 23: 9510 / 10000
Epoch 24: 9483 / 10000
Epoch 25: 9489 / 10000
Epoch 26: 9493 / 10000
Epoch 27: 9507 / 10000
Epoch 28: 9493 / 10000
Epoch 29: 9472 / 10000
CPU times: user 3min 27s, sys: 3.36 s, total: 3min 30s
Wall time: 3min 35s


In [ ]:
net.weights[1]

In [ ]:
import matplotlib.pyplot as plt
imshow(np.reshape(training_data[0][0], [28, 28]), cmap='Greys')